# DIGEL import

Tramite questo notebook vengono eseguiti dei controlli di qualità sui dati delle tabelle, il match tra nuovi termini e termini già nel digest (e.g. nomi di persone) e viene effettuato l'import. 

Inizialmente vengono segnalati eventuali **errori** (e.g. nomi di colonne errati). In seguito vengono visualizzate le liste di nomi di persone, riviste, editori e libri estratte dalle tabelle. Queste vanno ricontrollate manualmente affinchè non ci siano varianti dello stesso nome all'interno della tabella. Effettuati questi controlli, i nomi possono essere **riconciliati** (matching) con i nomi esistenti nel Digest online.

Quando i controlli e i match sono effettuati, i dati vengono trasformati e inviati al Digest online.

Se viene riscontrato un errore, il processo si ferma e viene chiesto di modificare la tabella, ricaricarla e ripetere il processo dall'inizio. 

**Per evitare problemi nella cronologia dei file, clicca su `Runtime`> `Disconnetti runtime ed eliminane i dati`).**


## 1. Controlli preliminari

1. Si possono caricare più file excel ma viene processato **un solo file excel** alla volta. Per caricare dati da più file bisogna fare refresh della pagina e ripetere le operazioni (cambiando il nome del file da processare quando richiesto)
2. Il file excel deve essere nel **formato** `.xlsx`
3. Le **colonne** della tabella devono rispettare le norme redazionali

### 1.1 Norme per la redazione delle tabelle

I nomi delle intestazioni delle colonne devono essere come segue. L'ordine non è importante, mentre è fondamentale che non ci siano colonne extra non elencate qui sotto.

**Nomi delle colonne**

 * reference (APA) (*opzionale*)
 * publication type	
 * title	
 * author_1	, author_2, author_3, author_4, author_5, author_6, author_7, author_8, author_9, author_10	
 * journal	
 * volume	
 * issue	
 * pages	
 * year	
 * main_theme	
 * secondary_theme	
 * lang	
 * link	
 * doi 
 * booktitle
 * publisher
 * editor_1, editor_2, editor_3, editor_4, editor_5, editor_6, editor_7, editor_8, editor_9, editor_10

**Vocabolari controllati**

*publication type*
 * 'Article',
 * 'Journal Issue',
 * 'Report'
 * 'Book Chapter',
 * 'Book',
 * 'Doctoral Thesis'
 * 'Grey Literature'

*lang*
 * "Polish"
 * "Spanish"
 * "Portuguese"
 * "Italian"
 * "Dutch"
 * "German"
 * "French"
 * "English" 

*main_theme* e *secondary_theme*

 * "community work", 
 * "educational partnerships", 
 * "formal education", 
 * "higher education research", 
 * "informal education including youth work", 
 * "international volunteering", 
 * "media", 
 * "non-formal education", 
 * "policy related research", 
 * "study visits", 
 * "teacher education", 
 * "theoretical", 
 * "training of trainers", 
 * "teaching material" (*solo in main_theme*)

## 2. Caricamento del file excel

1. Nella barra a sinistra, clicca sull'icona della cartella per espandere il menu 
2. Fare Drag and drop del file excel dal proprio computer alla sidebar (rilasciare il mouse nello spazio bianco, non nella cartella `sample_data`)

<img src="https://github.com/dharc-org/digestgce/raw/main/import.png" width="500px">

*Comparirà un alert in cui si ricorda che i dati verranno cancellati dal drive allo scadere della sessione (quando si esce da questa finestra)*

3. Nel menu in alto, clicca su `Runtime` e poi su `Esegui tutte`. Segui i prompt in cui ti viene chiesto di rivedere le liste e confermare.

<img src="https://github.com/dharc-org/digestgce/raw/main/run.png" width="500px">


In [ ]:
# install libraries
!pip install sparql-dataframe
!pip install rdflib

# import libraries
import requests
url = 'https://raw.githubusercontent.com/dharc-org/digestgce/main/digel.py'
r = requests.get(url)
with open('digel.py', 'w') as f:
    f.write(r.text)

import digel as dg
import hashlib
import re
import requests , urllib.parse
from collections import defaultdict
from datetime import datetime

import pandas as pd
import numpy as np

import sparql_dataframe
from rdflib import Graph, Literal, RDF, URIRef, ConjunctiveGraph, Namespace
from rdflib.namespace import XSD, OWL, RDF , RDFS, DCTERMS

from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 3. Controllo qualità

Segui le istruzioni e i prompt che ti vengono richiesti

In [ ]:
input_file = input("Inserisci il nome completo del file xlsx da processare e premi invio: ")

Inserisci il nome completo del file xlsx da processare e premi invio: italy.xlsx


In [ ]:
sheet_name = input("Inserisci il nome del tab con i dati e premi invio: ") 

Inserisci il nome del tab con i dati e premi invio: italy


In [ ]:
try:
  df = pd.read_excel(input_file,sheet_name=sheet_name)
except:
  raise TypeError("ERRORE - Il nome del file o del tab sono sbagliati. Fai refresch della pagina e ricomincia")

### 3.1 Controllo nomi colonne

In [ ]:
cols = list(df.columns.values)
cols = [x for x in cols if 'Unnamed' not in x]
dg.check_columns(cols, dg.COLUMNS, 'y')
# lowercase all columns
df.columns= df.columns.str.lower()

OK -- i nomi delle colonne sono corretti


### 3.2 Controllo vocabolari



In [ ]:
# lang
lang = df['lang'].dropna().unique().tolist()
dg.check_vocabulary(lang, dg.LANGUAGES, 'lang')
# pub type
pub_type = df['publication type'].dropna().unique().tolist()
dg.check_vocabulary(pub_type, dg.PUBLICATION_TYPES, 'publication type')
# primary subject
primary_type = df['main_theme'].dropna().unique().tolist()
dg.check_vocabulary(primary_type, dg.PRIMARY_SUBJECTS, 'main_theme','y')
# secondary subject
secondary_type = df['secondary_theme'].dropna().unique().tolist()
dg.check_vocabulary(secondary_type, dg.SECONDARY_SUBJECTS, 'secondary_theme','y')

OK -- il dizionario nella colonna lang è corretto
OK -- il dizionario nella colonna publication type è corretto
OK -- il dizionario nella colonna main_theme è corretto
OK -- il dizionario nella colonna secondary_theme è corretto


### 3.3 Controllo tipi di valori

TODO

### 3.4 Controllo omonimi

**Autori e editori**

Sotto viene riportata la lista di autori (valori unici). Ricontrolla la lista. 

Se ci sono valori erronei (e.g. "-"), valori doppi in un campo (e.g. "Costa, M.F. & Rodrigues") o doppioni (e.g. "Lima, M.J.M" e "Lima, M.J.M."), modifica il file excel, fai refresch della pagina, ricarica il file ed esegui tutto (Runtime > Esegui tutte).

In [ ]:
if 'editor_1' in df.columns:
  user_people = dg.check_omonyms([df['author_1'], df['author_2'],
                df['author_3'], df['author_4'], df['author_5'], df['author_6'],
                df['author_7'], df['author_8'], df['author_9'],df['author_10'],
                df['editor_1'], df['editor_2'],
                df['editor_3'], df['editor_4'], df['editor_5'], df['editor_6'],
                df['editor_7'], df['editor_8'], df['editor_9'],df['editor_10']])
else:
  user_people = dg.check_omonyms([df['author_1'], df['author_2'],
                df['author_3'], df['author_4'], df['author_5'], df['author_6'],
                df['author_7'], df['author_8'], df['author_9'],df['author_10']])

Ambel, Mario
Bonel, Roberta
Caccioppola, Federica
Caruso, Angela
Da Re, Lorenza
Dato, Daniela
De Toffoli, Matteo
Galligani, Ivan
Grion, Valentina
Ladogana, Manuela
Laneri, Calogero
Libro Bianco
Mungongo, Emmanuel Kasongo
Nuzzaci, Antonella
Panarello, Patrizia
Porcarelli, Andrea
Rizzi Paola
Simeone, Domenico
Tagliavia, Alfredo
Tino, Concetta
UNESCO
VV.AA.
WE WORLD
Zani, Vincenzo


In [ ]:
continue_parsing = input("Se i nomi delle persone sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i nomi delle persone sono corretti premi invio, altrimenti modifica il file e ricarica


**Journals**

Procedi come per gli autori

In [ ]:
user_journals = dg.check_omonyms([df['journal']])

In [ ]:
continue_parsing = input("Se i journal sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i journal sono corretti premi invio, altrimenti modifica il file e ricarica


**Publisher**

Procedi come per gli autori e i journal

In [ ]:
user_publishers = dg.check_omonyms([df['publisher']]) if 'publisher' in df.columns else {}

Edizioni ETS
Edizioni Sapienza
Falzea
LUMSA
Pensa MultiMedia
Piemme Edizioni
UNESCO
Università degli studi RomaTre
Vita e Pensiero Editrice
We World
Zeroseiup


In [ ]:
continue_parsing = input("Se i publisher sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i publisher sono corretti premi invio, altrimenti modifica il file e ricarica


**Book titles**

Procedi come per gli autori, i journal e i publisher. 

In [ ]:
user_books = dg.check_omonyms([df['booktitle']]) if 'booktitle' in df.columns else {}

CONFINI, IDENTITÀ E CITTADINANZA Uno sguardo multidisciplinare
Cittadinanza europea, Costituzione e Diritti: l'educazione come strumento di democrazia. Il progetto #ShareEU
Una scuola per la cittadinanza


In [ ]:
continue_parsing = input("Se i titoli di libro sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i titoli di libro sono corretti premi invio, altrimenti modifica il file e ricarica


## 4. Riconciliazione con i termini online

Viene eseguito il match tra i termini delle liste appena esaminate (autori e journals) e quelli presenti online. 

N.B. Al momento il match è preciso (le stringhe devono combaciare).

Per i termini nuovi, verrà creato un nuovo dato (una nuova persona o un nuovo journal). Viene riportato sotto con la dicitura: 

`NEW -- Dante Alighieri`

Per i termini che hanno un match, viene riportata la URL che identifica la persona, e.g. 

`MATCH -- Tarozzi, Massimiliano -- https://projects.dharc.unibo.it/digestgel/term-1653893507-239157`

I match vanno ricontrollati manualmente. Se un match è errato (sono omonimi ma non sono la stessa persona o rivista) va cambiato il nome nella tabella, oppure si tiene traccia del nome della persona e si cambia online.

### 4.1 Match persone

In [ ]:
df_people = dg.get_entities('people')
match_people = dg.match_entities(user_people,df_people['author'], df_people['authorURI'])

NEW -- Ambel, Mario
NEW -- Bonel, Roberta
NEW -- Caccioppola, Federica
NEW -- Caruso, Angela
NEW -- Da Re, Lorenza
NEW -- Dato, Daniela
NEW -- De Toffoli, Matteo
NEW -- Galligani, Ivan
NEW -- Grion, Valentina
NEW -- Ladogana, Manuela
NEW -- Laneri, Calogero
NEW -- Libro Bianco
NEW -- Mungongo, Emmanuel Kasongo
NEW -- Nuzzaci, Antonella
NEW -- Panarello, Patrizia
NEW -- Porcarelli, Andrea
NEW -- Rizzi Paola
MATCH -- Simeone, Domenico : https://projects.dharc.unibo.it/digestgel/term-1671008214-83892
NEW -- Tagliavia, Alfredo
NEW -- Tino, Concetta
MATCH -- UNESCO : https://projects.dharc.unibo.it/digestgel/term-1653893507-239402
MATCH -- VV.AA. : https://projects.dharc.unibo.it/digestgel/term-1653893507-239467
NEW -- WE WORLD
NEW -- Zani, Vincenzo


In [ ]:
continue_parsing = input("Se i match sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i match sono corretti premi invio, altrimenti modifica il file e ricarica


### 4.2 Match journals

In [ ]:
df_journals = dg.get_entities('journals')
match_journals = dg.match_entities(user_journals,df_journals['journal'], df_journals['journalURI'])

In [ ]:
continue_parsing = input("Se i match sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i match sono corretti premi invio, altrimenti modifica il file e ricarica


### 4.3 Match publishers

In [ ]:
df_publishers = dg.get_entities('publishers')
match_publishers = dg.match_entities(user_publishers,df_publishers['publisher'], df_publishers['publisherURI'])

NEW -- Edizioni ETS
NEW -- Edizioni Sapienza
NEW -- Falzea
NEW -- LUMSA
MATCH -- Pensa MultiMedia : https://projects.dharc.unibo.it/digestgel/term-1653893507-245298
NEW -- Piemme Edizioni
MATCH -- UNESCO : https://projects.dharc.unibo.it/digestgel/term-1653893507-2455
NEW -- Università degli studi RomaTre
NEW -- Vita e Pensiero Editrice
NEW -- We World
NEW -- Zeroseiup


In [ ]:
continue_parsing = input("Se i match sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i match sono corretti premi invio, altrimenti modifica il file e ricarica


### 4.4 Match Book titles

In [ ]:
df_books = dg.get_entities('books')
match_books = dg.match_entities(user_books,df_books['book'], df_books['bookURI'])

NEW -- CONFINI, IDENTITÀ E CITTADINANZA Uno sguardo multidisciplinare
NEW -- Cittadinanza europea, Costituzione e Diritti: l'educazione come strumento di democrazia. Il progetto #ShareEU
NEW -- Una scuola per la cittadinanza


In [ ]:
continue_parsing = input("Se i match sono corretti premi invio, altrimenti modifica il file e ricarica")

Se i match sono corretti premi invio, altrimenti modifica il file e ricarica


## 5. Trasformazione dati e import

I dati tabulari vengono trasformati nel formato usato dal digest online e vengono caricati automaticamente.

Viene richiesta la password per autenticare l'import.

In [ ]:
api_key = input("inserisci la password per l'import:" )

inserisci la password per l'import:digestIMPORT!


In [ ]:
confirm = input("Confermi che possiamo procedere all'import? premi invio se sì" )

Confermi che possiamo procedere all'import? premi invio se sì


In [ ]:
authors_uri, journals_uri, publishers_uri, books_uri = dg.prepare_uris(user_people, user_journals, user_publishers, user_books)
dg.create_graphs(df,
                 authors_uri, journals_uri, publishers_uri, books_uri, 
                 api_key, 
                 match_people, match_journals, match_publishers, match_books)

sending request for 1671102923-882837
API URL: https://projects.dharc.unibo.it/digestgel/import
request sent 1671102923-882837 b'@prefix biro: <http://purl.org/spar/biro/> .\n@prefix fabio: <http://purl.org/spar/fabio/> .\n@prefix gce: <https://w3id.org/digestgel/> .\n@prefix ns1: <http://purl.org/dc/terms/> .\n@prefix ns2: <http://dbpedia.org/ontology/> .\n@prefix prism: <http://prismstandard.org/namespaces/basic/2.0/> .\n@prefix prov: <http://www.w3.org/ns/prov#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\ngce:1671102923-882837 a fabio:Book ;\n    rdfs:label "Educare alla cittadinanza locale e globale. Contesti, percorsi, esperienze" ;\n    prism:publicationDate <https://w3id.org/digestgel/vocabularies/2021> ;\n    ns1:language <https://w3id.org/digestgel/vocabularies/italian> ;\n    ns1:publisher gce:1671102923-870619 ;\n    ns1:title "Educare alla cittadinanza locale e globale. Contesti, percorsi, esperienze" ;\n

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.head()


,publication type,reference (apa),lang,title,booktitle,author_1,author_2,author_3,author_4,author_5,...,year,main_theme,secondary_theme,publisher,journal,volume,issue,pages,link,doi
0,Book,"Dato, D., & Ladogana, M. (A cura di). (2021). ...",Italian,Educare alla cittadinanza locale e globale. Co...,NaN,"Dato, Daniela","Ladogana, Manuela",NaN,NaN,NaN,...,2021,teacher education,training of trainers,Zeroseiup,NaN,NaN,NaN,NaN,https://www.zeroseiup.eu/prodotto/educare-alla...,NaN
1,Book,"Mungongo, E.K. (2021). Educazione alla cittadi...",Italian,Educazione alla cittadinanza globale: Focus su...,NaN,"Mungongo, Emmanuel Kasongo",NaN,NaN,NaN,NaN,...,2021,conceptual publications,theoretical,Edizioni Sapienza,NaN,NaN,NaN,NaN,https://www.miur.gov.it/documents/20182/439463...,NaN
2,Book,"Panarello, P. (2020). In difesa del pianeta vi...",Italian,In difesa del pianeta vivente per i diritti di...,NaN,"Panarello, Patrizia",NaN,NaN,NaN,NaN,...,2020,conceptual publications,theoretical,Falzea,NaN,NaN,NaN,NaN,https://sol.unibo.it/SebinaOpac/resource/in-di...,NaN
3,Book,"Ambel, Mario. (A cura di). (2020). Una scuola ...",Italian,"Una scuola per la cittadinanza. Idee, Percorsi...",NaN,"Ambel, Mario",NaN,NaN,NaN,NaN,...,2020,teacher education,training of trainers,Piemme Edizioni,NaN,NaN,NaN,NaN,NaN,NaN
4,Book,"Simeone, D., & Zani, V. (A cura di). (2021). L...",Italian,La casa della pace. Un progetto educativo in d...,NaN,"Simeone, Domenico","Zani, Vincenzo",NaN,NaN,NaN,...,2021,community work,informal education including youth work,Vita e Pensiero Editrice,NaN,NaN,NaN,NaN,https://www.vitaepensiero.it/scheda-libro/auto...,NaN
